# DS-NYC-45 | Unit Project 3: Basic Machine Learning Modeling

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [2]:
df = pd.read_csv(os.path.join('..', 'dataset', 'ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether or not an applicant was admitted.

In [3]:
# TODO
print pd.crosstab(df['admit'],df['prestige'], rownames=['admit'])

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


## Part B.  Variable Transformations

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [4]:
# TODO
pd.get_dummies(df['prestige'])

,1.0,2.0,3.0,4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
395,0.0,1.0,0.0,0.0
396,0.0,0.0,1.0,0.0
397,0.0,1.0,0.0,0.0
398,0.0,1.0,0.0,0.0


> ### Question 3.  How many of these binary variables do we need for modeling?

### Answer: 3

> ### Question 4.  Why are we doing this?

Answer: We only need 3 because the fourth is redundant.

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [5]:
df_prestige1 = pd.get_dummies(df.prestige, drop_first=True)
df_prestige1

,2.0,3.0,4.0
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,0.0,1.0
...,...,...,...
395,1.0,0.0,0.0
396,0.0,1.0,0.0
397,1.0,0.0,0.0
398,1.0,0.0,0.0


In [7]:
# TODO
df_prestige=pd.get_dummies(df['prestige'])

In [8]:
df = df.join(df_prestige)

In [9]:
df= df.drop('prestige', axis=1)

In [10]:
df.head()

,admit,gre,gpa,1.0,2.0,3.0,4.0
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether or not an applicant was admitted.

In [173]:
# TODO
print pd.crosstab(df['admit'],df[1.0], rownames=['admit'])

1.0    0.0  1.0
admit          
0      243   28
1       93   33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [174]:
# TODO
33.0/28

1.1785714285714286

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [175]:
# TODO
93.0/243

0.38271604938271603

> ### Question 9.  Finally, what's the odds ratio?

In [176]:
# TODO
1.17/0.38

3.078947368421052

> ### Question 10.  Write this finding in a sentenance.

Answer:For students who attended top ranked college, the odds of being admitted into graduate school is 3 times the odds of students who did not attend top ranked college.

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentenance.

In [177]:
# TODO
print pd.crosstab(df['admit'],df[4.0], rownames=['admit'])

4.0    0.0  1.0
admit          
0      216   55
1      114   12


(12.0/55)/(114.0/216) For students who attended least ranked college, the odds of being admitted into graduate school is 0.4 times the odds of students who attended to better ranked college.

## Part C. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model prediting admission into UCLA using `gre`, `gpa`, and the prestige of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [178]:
df.head()

,admit,gre,gpa,1.0,2.0,3.0,4.0
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


In [179]:
# TODO
train_cols = ['gre','gpa',2.0,3.0,4.0]
logit = smf.Logit(df['admit'], df[train_cols])
result = logit.fit()
#train_cols = df.columns - ['admit', 1.0]

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


> ### Question 13.  Print the model's summary results.

In [180]:
# TODO
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Sun, 29 Jan 2017   Pseudo R-squ.:                 0.05722
Time:                        17:33:47   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.308      0.191        -0.001     0.003
gpa           -0.1323      0.195     -0.680      0.497        -0.514     0.249
2.0           -0.9562      0.302     -3.171      0.0

> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [181]:
# TODO
print result.conf_int()

            0         1
gre -0.000680  0.003414
gpa -0.513657  0.249045
2.0 -1.547279 -0.365166
3.0 -2.188769 -0.886230
4.0 -2.656743 -1.083112


In [182]:
# odds ratios only
print np.exp(result.params)

gre    1.001368
gpa    0.876073
2.0    0.384342
3.0    0.214918
4.0    0.154135
dtype: float64


In [183]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

         2.5%     97.5%        OR
gre  0.999320  1.003420  1.001368
gpa  0.598303  1.282800  0.876073
2.0  0.212826  0.694082  0.384342
3.0  0.112055  0.412207  0.214918
4.0  0.070176  0.338540  0.154135


> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: We can expect the odds of being admitted graduate school to decrease by about 1% if the prestige of a school is 2. 

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer: We can expect the odds of being admitted graduate school to increase by about 200% if the applicant's GPA is higher.

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO
result.predict((800,4,1,0,0,0))

In [ ]:
result.predict((800,4,0,1,0,0))

In [ ]:
result.predict((800,4,0,0,1,0))

In [ ]:
result.predict((800,4,0,0,0,1))

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler 
Scaler =StandardScaler()
X=Scaler.fit_transform(X)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# TODO
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression(C=10**2)
#X=X_train[df.columns[1:]]
#y=y_train['admit']
output=LR.fit(X_train,y_train)

In [188]:
preds = output.predict(X_test)

In [189]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,preds)
print "LR accuracy is {:.2f}".format(score)


LR accuracy is 0.57


> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [190]:
# TODO
np.exp(result.params)


gre    1.001368
gpa    0.876073
2.0    0.384342
3.0    0.214918
4.0    0.154135
dtype: float64

In [192]:
np.exp(LR.coef_)

array([[ 1.3947473 ,  1.4441099 ,  1.32302668,  1.22144451,  0.78292452,
         0.79691981]])

Answer:

> ### Question 20.  Again assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO

Answer: